In [ ]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import phik

pd.options.display.max_columns = None #полностью отображаем столбцы
warnings.filterwarnings("ignore")
sns.set_style("whitegrid")

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("arnabchaki/data-science-salaries-2023")

print("Path to dataset files:", path)

In [ ]:
df = pd.read_csv('C:/Users/Public/Documents/ds_salaries.csv')
df.info()

In [ ]:
df.head(10)

In [ ]:
#заменим верхний регистр на нижний
df.columns = df.columns.str.lower()
df.columns

In [ ]:
#  Проверка пропущенных значений

df.isna().sum()


In [ ]:
# заполнение пустых городов

df['employee_residence'].fillna('Unknown', inplace=True)

In [ ]:
# Проверка и удаление дубликатов

df.duplicated().sum()  # Количество дубликатов

# Удаление дубликатов
df = df.drop_duplicates()

In [ ]:
# Ещё раз проверим

df.isna().sum()
df.duplicated().sum()

In [ ]:
#  1. Самые высокооплачиваемые должности

df.groupby('job_title')['salary_in_usd'].median().sort_values(ascending=False).head(10)

In [ ]:
# 2. Есть ли выбросы в зарплатах? В каких должностях?

plt.figure(figsize=(15, 6))
sns.boxplot(data=df, x='job_title', y='salary_in_usd')
plt.xticks(rotation=90)
plt.title("Выбросы зарплаты по должностям")
plt.show()

In [ ]:
# Влияет ли удалёнка на зарплату?

df.groupby('remote_ratio')['salary_in_usd'].median()

# (0 = нет удалёнки, 50 = гибрид, 100 = полностью удалённо)

In [ ]:
# 4. Зависит ли зарплата от размера компании?

df.groupby('company_size')['salary_in_usd'].median().sort_values(ascending=False)

In [ ]:
# 5. Популярные страны и уровень зарплат

# Популярные

df['employee_residence'].value_counts().head(10)



In [ ]:
# Средняя зарплата по странам

df.groupby('employee_residence')['salary_in_usd'].median().sort_values(ascending=False).head(10)

In [ ]:
#6. Медианная зарплата по должностям

df.groupby('job_title')['salary_in_usd'].median().sort_values(ascending=False)

In [ ]:
# 7. Как изменились зарплаты в 2023 по сравнению с 2020

# Медианная зарплата по должностям в 2020 и 2023
# Оставим только 2020 и 2023
salary_filtered = df[df['work_year'].isin([2020, 2023])]

# Группируем по году и должности → медианная зарплата
salary_by_year = salary_filtered.groupby(['job_title', 'work_year'])['salary_in_usd'].median().unstack()

# Убедимся, что обе колонки (2020 и 2023) есть
salary_by_year = salary_by_year.dropna(subset=[2020, 2023])

# Добавим процент изменения
salary_by_year['change_%'] = ((salary_by_year[2023] - salary_by_year[2020]) / salary_by_year[2020]) * 100

# Сортировка по убыванию прироста
salary_by_year.sort_values('change_%', ascending=False)



In [ ]:
 #8. Портрет самого успешного работника

df[df['salary_in_usd'] == df['salary_in_usd'].max()]

In [ ]:
#9. Живут и работают в разных странах

# Люди, у кого место работы и проживания не совпадает
df_diff_country = df[df['employee_residence'] != df['company_location']]


In [ ]:
# Количество таких людей
df_diff_country.shape[0]


In [ ]:
# Сколько процентов работают на другую страну
percent_remote_country = df_diff_country.shape[0] / df.shape[0] * 100
percent_remote_country